In [ ]:
import time
t1 = time.time()

In [ ]:
# import matplotlib.pyplot as plt
from datetime import date
import pandas as pd
import itertools
import numpy as np
import os

from configparser import ConfigParser
import pyodbc

# import statsmodels as sm
import pmdarima as pm
# from pmdarima.model_selection import train_test_split
# from sklearn.preprocessing import LabelBinarizer



### Settings

In [ ]:
# Paths & Files
db_conf_path = os.path.join(os.environ['USERPROFILE'],'db_connection.conf')

# ML Settings
# testsize = .05
scale='B'                # Check if needed or hardcoded
periods = 5              # Check if needed or hardcoded
weighted_total = False
prediction_length = 5

# Data Settings
myfilters = {'SITE':['VELIZY'],
             'SERVICE_ACTIVITE':['PRESTATION','CONTRAT - COTISATION']}

#### DB Connection

In [ ]:
def db_connection(db_conf):
    config = ConfigParser()
    config.read(db_conf_path)

    srv_name = config['CONNECTION']['srv_name']
    db_name = config['CONNECTION']['db_name']
    drv = config['CONNECTION']['drv']
    usr = config['CONNECTION']['usr']
    pwd = config['CONNECTION']['pwd']

    conn_str = 'DRIVER=%s; SERVER=%s;DATABASE=%s;UID=%s;PWD=%s' % (drv, srv_name, db_name, usr, pwd)
    cnxn = pyodbc.connect(conn_str)
    return cnxn

#### Get Datas

In [ ]:
def get_datas(strsql):
    with db_connection(db_conf_path) as conn:
        result = pd.read_sql(sqlstr, conn)
    return result

#### Preprocess columns

In [ ]:
def preprocess_df(df,weighted):
    if weighted:
        df['TOTAL'] = df.TOTAL * df.COEFF
    df = df.drop(columns=['COEFF'])
    return df

#### Create Aggregated Df with DateTime Index

In [ ]:
def aggregate_df(df):
    df2 = df.groupby('DATEDATA').sum()
    df2 = df2.asfreq(freq='B')
    df2['TOTAL'].fillna(0,inplace=True)
    df2=df2.resample(scale).sum()
    return df2

### Main Loop Iterate over filters

In [ ]:
# Generate loops variants
myfilters = {'SITE':['VELIZY',],
             'SERVICE_ACTIVITE':['PRESTATION','CONTRAT - COTISATION']}

v_keys = list(myfilters.keys())
v = list(myfilters.values())

liste = list(itertools.product(*v))

In [ ]:
strSQL_start = "SELECT * FROM ML_IGE_ENTRANTS_DS1 "

for variant in liste:
    print(variant)

    # Generate SELECT query
    strSQLW = "WHERE "
    for i in range(len(variant)):
        strSQLW += v_keys[i]
        strSQLW += "="
        strSQLW += "'" + str(variant[i]) + "'"
        strSQLW += ' AND '
    strSQL = strSQL_start + strSQLW[:-5]
    
    # Get datas
    df = get_datas(strSQL)
    
    # Prepare datas
    df_pre = preprocess_df(df, weighted_total)
    df_agg = aggregate_df(df_pre)
    
    # ARIMA Modeling
    model = pm.auto_arima(df_agg, error_action='ignore', trace=False,
                          suppress_warnings=True, maxiter=10,
                          seasonal=True, m=periods)
    preds, conf_int = model.predict(n_periods=prediction_length, return_conf_int=True)
    
    # Create Prediction dataframe to write in DB
    idx=[]
    for i in range(1,prediction_length+1):
        nd = df_agg.index[-1]+pd.tseries.offsets.BDay(i)
        idx.append(nd)
    t=pd.DataFrame(np.concatenate((preds.reshape(-1,1),conf_int),axis=1),columns=['PRED','LB','UB'],index=idx)

    for i in range(len(variant)):
        t[v_keys[i]]=variant[i]
    
    t['WEIGHTED']=weighted_total
    today = date.today()
    d = today.strftime("%Y-%m-%d")
    t['PREDDATE']=d
    
    
    # Create DELETE SQL string
    strSQL = "DELETE * FROM ML_IGE_ENTRANTS_PRED WHERE DATEPRED=\'" + d + "\'"

    # with db_connection(db_conf_path) as conn:
    #     conn.execute(strSQL)
    #     conn.commit()
    
    
    # Create INSERT SQL string
    strSQL = "INSERT INTO ML_IGE_ENTRANTS_PRED (DATEDATA,PRED,LB,UB,SITE,SERVICE_ACTIVITE,WEIGHTED,DATEPRED) VALUES "
    values=""
    for idx, row in t.iterrows():
        values += "("
        values += "\'" + idx.strftime('%Y-%m-%d') + "\',"
        values += str(int(row.PRED)) + ','
        values += str(int(row.LB)) + ','
        values += str(int(row.UB)) + ','
        values += row.SITE + ','
        values += row.SERVICE_ACTIVITE + ','
        values += str(int(row.WEIGHTED)) + ','
        values += "\'" + row.PREDDATE + "\'"
        values+='),'

    strSQL += values
    strSQL = strSQL[:-1]

    # with db_connection(db_conf_path) as conn:
    #     conn.execute(strSQL)
    #     conn.commit()

In [ ]:
t

In [ ]:
tt

In [ ]:
tt = t.reset_index(level=0)
tt.rename(columns={'index':'DATEDATA'},inplace=True)
tt['DATEDATA'] = [x.strftime('%Y-%m-%d') for x in tt['DATEDATA']]
print(tuple(list(tt.columns)))
str(tuple(list(tt.iloc[0])))

In [ ]:
for idx,row in tt.iterrows():
    print(tuple((row)))

In [ ]:
df = get_datas(sqlstr)
# df

In [ ]:
df_pre = preprocess_df(df, weighted_total)
# df_pre

In [ ]:
df_agg = aggregate_df(df_pre)
# df_agg

In [ ]:
model = pm.auto_arima(df_agg, error_action='ignore', trace=False,
                      suppress_warnings=True, maxiter=10,
                      seasonal=True, m=periods)

In [ ]:
# print(model.order)
# print(model.seasonal_order)
# plt.rcParams['figure.figsize'] = [10, 10]
# model.plot_diagnostics();

In [ ]:
preds, conf_int = model.predict(n_periods=prediction_length, return_conf_int=True)

In [ ]:
# Create Prediction dataframe to write in DB
idx=[]
for i in range(1,prediction_length+1):
    nd = df_agg.index[-1]+pd.tseries.offsets.BDay(i)
    idx.append(nd)
t=pd.DataFrame(np.concatenate((preds.reshape(-1,1),conf_int),axis=1),columns=['PRED','LB','UB'],index=idx)

t['SITE']='VELIZY'                       # PARAMETER
t['SERVICE_ACTIVITE']='PRESTATION'       # PARAMETER
t['WEIGHTED']=weighted_total

today = date.today()
d = today.strftime("%Y-%m-%d")
t['PREDDATE']=d

In [ ]:
print(df_agg)

In [ ]:
# Delete predictions if existing
strSQL = "DELETE * FROM ML_IGE_ENTRANTS_PRED WHERE DATEPRED=\'" + d + "\'"

# with db_connection(db_conf_path) as conn:
#     conn.execute(strSQL)
#     conn.commit()

In [ ]:
strSQL

In [ ]:
# Create INSERT SQL string

strSQL = "INSERT INTO ML_IGE_ENTRANTS_PRED (DATEDATA,PRED,LB,UB,SITE,SERVICE_ACTIVITE,WEIGHTED,DATEPRED) VALUES "
values=""
for idx, row in t.iterrows():
    values += "("
    values += "\'" + idx.strftime('%Y-%m-%d') + "\',"
    values += str(int(row.PRED)) + ','
    values += str(int(row.LB)) + ','
    values += str(int(row.UB)) + ','
    values += row.SITE + ','
    values += row.SERVICE_ACTIVITE + ','
    values += str(int(row.WEIGHTED)) + ','
    values += "\'" + row.PREDDATE + "\'"
    values+='),'

strSQL += values
strSQL = strSQL[:-1]

# with db_connection(db_conf_path) as conn:
#     conn.execute(strSQL)
#     conn.commit()

In [ ]:
strSQL

### Plot Result

In [ ]:
graph = pd.DataFrame(test_endog.copy())
graph = graph.rename(columns={'TOTAL':'TEST'})
graph['preds'] = preds
# graph['hwpreds'] = hwpreds
graph['lb'] = conf_int[:,0]
graph['ub'] = conf_int[:,1]
graph = graph.append(pd.DataFrame(train_endog))

In [ ]:
graphz = graph[graph.index>'2020-01-01']

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
g1 = plt.plot(graphz.index, graphz.TOTAL, label='Train')
g2 = plt.plot(graphz.index, graphz.TEST,'c--', label='Test')
g3 = plt.plot(graphz.index, graphz.preds,'r', label='Pred',linewidth=2, alpha=.5)
# g3b = plt.plot(graphz.index, graphz.hwpreds,'g', label='Pred',linewidth=2)
g4 = plt.fill_between(graphz.index,graphz.lb,graphz.ub,color='r',alpha=.1, label='C.I.')
plt.legend(loc='upper left')
plt.title('Actual test samples vs. forecasts')
plt.show()

In [ ]:
graphz

In [ ]:
t2 = time.time()

print('Total Run time: %i secs' % (t2-t1))